# Setup
We need datasets, dataloaders, and code to train and evaluate models

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

KeyboardInterrupt: 

In [ ]:
import os
import sys
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)
from models import *
from data_structures import *
from helpers import load_from_hdf5

In [ ]:
X, y = load_from_hdf5(filename="../../sample_windows.h5")

# 1d CNN
First, let's try a one dimensional CNN to capture temporal relationships within each feature.

In [ ]:
train_loader, test_loader = create_train_test_dataloaders(X, y)

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [ ]:
num_feats = 44
model = CNNRULRegression(num_features=num_feats).to(device)
criterion = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 20 

history = train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)

In [ ]:
plot_loss(history)

In [ ]:
plot_rul_predictions(model, test_loader, device)

# 2d CNN
Now, let's try out a two dimensional CNN to capture interactions among features.

In [ ]:
train_loader, test_loader = create_train_test_dataloaders(X, y, dim="2d")

In [ ]:
num_feats = 44
model = CNNRUL2DRegression(num_features=num_feats).to(device)
criterion = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 15 

history = train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)

In [ ]:
plot_loss(history)

In [ ]:
plot_rul_predictions(model, test_loader, device)

# Hybrid CNN
Now, let's try a hybrid CNN that uses both 1d and 2d convolutions.

In [ ]:
train_loader, test_loader = create_train_test_dataloaders(X, y)

In [ ]:
num_feats = 44
seq_length = 50  
model = HybridCNNRegression(num_features=num_feats, seq_length=seq_length).to(device)
criterion = nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 15

history = train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)

In [ ]:
plot_loss(history)

In [ ]:
plot_rul_predictions(model, test_loader, device)

# More Complex Hybrid Model
The model's loss dropping drastically and then remaining stable might indicate underfitting. So, let's try adding additional convolutional layers.

In [ ]:
num_feats = 44
seq_length = 50  
model = ComplexHybridCNNRegression(num_features=num_feats, seq_length=seq_length).to(device)
criterion = nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 15

history = train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)

In [ ]:
plot_loss(history)

In [ ]:
plot_rul_predictions(model, test_loader, device)

# Temporal CNN
Our more complex hybrid CNN was massively overfitting, so let's go in a different direction. Let's try a temporal CNN instead. 

In [ ]:
num_feats = 44
seq_length = 50
model = TCNRegression(num_features=num_feats, seq_length=seq_length).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 20

history = train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)
plot_loss(history)

In [ ]:
plot_rul_predictions(model, test_loader, device)